In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import getpass

username = getpass.getuser()

In [2]:
spark = SparkSession.\
        builder.\
        config('spark.ui.port','0').\
        config('spark.sql.warehouse.dir',f'/user/{username}/warehouse').\
        config('spark.shuffle.useOldFetchProtocol','true').\
        enableHiveSupport().\
        master('yarn').\
        getOrCreate()


In [3]:
loans_default_schema = "member_id string ,delinq_2yrs float ,delinq_amount float ,public_record float,public_record_bankruptcies float ,inquiries_last_6months float,total_received_late_fee float,months_since_last_delinq float,months_since_last_record float"

In [4]:
loans_default_raw_df = spark .read.csv("/user/itv015278/lendingclubproject/raw/loans_defaulters_csv",header = True , schema = loans_default_schema) 

In [5]:
loans_default_deling_updated_df = loans_default_raw_df.withColumn("delinq_2yrs" , col("delinq_2yrs").cast("int")).fillna(0,subset=["delinq_2yrs"])
loans_default_deling_updated_df = loans_default_deling_updated_df.withColumn("public_record" , col("public_record").cast("int")).fillna(0,subset=["public_record"])
loans_default_deling_updated_df = loans_default_deling_updated_df.withColumn("public_record_bankruptcies" , col("public_record_bankruptcies").cast("int")).fillna(0,subset=["public_record_bankruptcies"])
loans_default_deling_updated_df = loans_default_deling_updated_df.withColumn("inquiries_last_6months" , col("inquiries_last_6months").cast("int")).fillna(0,subset=["inquiries_last_6months"])

In [6]:
loans_default_deling_updated_df.createOrReplaceTempView("loan_defaulters")

In [7]:
loans_def_delinq_df = spark.sql("select member_id , delinq_2yrs , delinq_amount , int(months_since_last_delinq) from loan_defaulters where delinq_2yrs > 0 or months_since_last_delinq > 0 ")

In [8]:
loans_def_inqueries_members_df = spark.sql("select member_id from loan_defaulters where public_record > 0.0 or public_record_bankruptcies > 0.0 or inquiries_last_6months > 0.0")

In [9]:
loans_def_inqueries_detail_df = spark.sql("select member_id ,public_record,public_record_bankruptcies,inquiries_last_6months  from loan_defaulters ")

In [11]:
loans_def_inqueries_detail_df.count()

2260701

In [12]:
loans_def_delinq_df.write.mode("overwrite").option("path","/user/itv015278/lendingclubproject/cleaned/loans_defaulters_records_delinq_parquet").save()

In [13]:
loans_def_inqueries_members_df.write.mode("overwrite").option("path","/user/itv015278/lendingclubproject/cleaned/loans_defaulters_records_inqueries_members_parquet").save()

In [14]:
loans_def_inqueries_detail_df.write.mode("overwrite").option("path","/user/itv015278/lendingclubproject/cleaned/loans_defaulters_records_inqueries_details_parquet").save()

In [2]:
#! hadoop fs -rm -R /user/itv015278/lendingclubproject/cleaned/loans_defaulters_records_inqueries_parquet

2025-01-02 06:03:25,038 INFO fs.TrashPolicyDefault: Moved: 'hdfs://m01.itversity.com:9000/user/itv015278/lendingclubproject/cleaned/loans_defaulters_records_inqueries_parquet' to trash at: hdfs://m01.itversity.com:9000/user/itv015278/.Trash/Current/user/itv015278/lendingclubproject/cleaned/loans_defaulters_records_inqueries_parquet


In [37]:
! hadoop fs -ls -h /user/itv015278/lendingclubproject/cleaned/loans_defaulters_records_inqueries_details_parquet

Found 3 items
-rw-r--r--   3 itv015278 supergroup          0 2025-01-02 06:12 /user/itv015278/lendingclubproject/cleaned/loans_defaulters_records_inqueries_details_parquet/_SUCCESS
-rw-r--r--   3 itv015278 supergroup     73.5 M 2025-01-02 06:12 /user/itv015278/lendingclubproject/cleaned/loans_defaulters_records_inqueries_details_parquet/part-00000-6713ca31-b086-412f-8248-4bffca7c45b1-c000.snappy.parquet
-rw-r--r--   3 itv015278 supergroup     70.8 M 2025-01-02 06:12 /user/itv015278/lendingclubproject/cleaned/loans_defaulters_records_inqueries_details_parquet/part-00001-6713ca31-b086-412f-8248-4bffca7c45b1-c000.snappy.parquet
